# Criando modelos customizados com sktime

O sktime oferece um ecossistema robusto, mas em cenarios reais frequentemente precisamos ajustar comportamentos, incorporar dados hierarquicos ou adicionar pre-processamentos especificos.  Felizmente, o sktime torna relativamente simples a criacao de modelos customizados, desde que sigamos algumas regras.

Acredito que essa e uma das grandes vantagens da biblioteca: o foco em ser extensivel e customizavel.

## O sistema de tags

Entre a chamada dos metodos publicos e privados existe uma camada de validacoes e conversões controlada pelas tags. Elas sinalizam ao `BaseForecaster` e ao `BaseTransformer` o que precisa ser garantido antes de executar a implementacao customizada.

As tags mais importantes para um forecaster podem ser definidas assim:

```python
_tags = {
	"capability:exogenous": True,
	"requires-fh-in-fit": False,
	"X_inner_mtype": [
		"pd.Series",
		"pd.DataFrame",
		"pd-multiindex",
		"pd_multiindex_hier",
	],
	"y_inner_mtype": [
		"pd.Series",
		"pd.DataFrame",
		"pd-multiindex",
		"pd_multiindex_hier",
	]
}
```

Cada uma delas indica o que o modelo é capaz de fazer nos seus métodos privados
`_fit` e `_predict`:

* `capability:exogenous`: Indica se o modelo suporta variáveis exógenas (X) durante o ajuste e a previsão.
* `requires-fh-in-fit`: Indica se o modelo precisa do horizonte de previsão (fh) durante o ajuste. Alguns modelos precisam devido a sua implementação interna.
* `y_inner_mtype`: Define os tipos de dados aceitos para a variável dependente (y) durante o ajuste e a previsão.
* `X_inner_mtype`: Define os tipos de dados aceitos para as variáveis exógenas (X) durante o ajuste e a previsão.
  
Os machine-types (**mtypes**), ou tipos para máquina, são a peça mais crucial nesse sistema.

### Machine-types disponiveis

Os `mtypes` definem qual a estrutura de dados que o modelo aceita como entrada e produz como saída. Os principais mtypes para séries temporais são:

- `np.ndarray`
- `pd.Series`
- `pd.DataFrame`
- `pd-multiindex` (ideia de painel)
- `pd_multiindex_hier` (dados hierarquicos)

Se o modelo suporta um `mtype` hierárquico e passamos um dado hierárquico, o 
dado chegará normalmente ao método privado `_fit` ou `_predict`. Caso contrário, o sktime tentará converter o dado para um mtype suportado.

#### Baixando exemplos por mtype

Para entender melhor cada mtype, podemos baixar exemplos práticos usando a função `get_examples` do sktime:

In [ ]:
from sktime.datatypes import get_examples

get_examples(mtype="np.ndarray", as_scitype="Series")[0]

In [ ]:
get_examples(mtype="pd.DataFrame", as_scitype="Series")[0].head()

In [ ]:
get_examples(mtype="pd-multiindex", as_scitype="Panel")[0].head()

In [ ]:
get_examples(mtype="pd_multiindex_hier", as_scitype="Hierarchical")[0].head()

Alguns mtypes tem limitações: uma `pd.Series` simples nao representa problemas hierarquicos, sendo necessario recorrer ao `pd_multiindex_hier`.

#### Exemplo prático

Vamos criar o nosso primeiro esqueleto de forecaster customizado. Para isso, baixamos uma série de exemplo:

In [ ]:
# | echo: false
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from sktime.forecasting.base import BaseForecaster
from sktime.utils._testing.series import _make_series

y = _make_series(4)
y

Nosso protótipo irá apenas printar os dados recebidos no método `_fit`. O `__init__` recebe um dicionário de tags para definir as capacidades do modelo.

In [ ]:
class Logger(BaseForecaster):

    _tags = {
        "requires-fh-in-fit": False,
    }

    def __init__(self, tags_to_set):
        self.tags_to_set = tags_to_set
        super().__init__()

        self.set_tags(**tags_to_set)
    
    def _fit(self, y, X=None, fh=None):
        print("Inside fit:")
        print(y)
        return self

In [ ]:
logger = Logger(tags_to_set={"y_inner_mtype" : ["pd.Series"] })
logger.fit(y)

In [ ]:
logger = Logger(tags_to_set={"y_inner_mtype" : ["np.ndarray"] })
logger.fit(y)

In [ ]:
logger = Logger(tags_to_set={"y_inner_mtype" : ["pd.DataFrame"] })
logger.fit(y)

In [ ]:
try:
    logger = Logger(tags_to_set={"y_inner_mtype" : ["pd_multiindex_hier"] })
    logger.fit(y)
except ValueError as e:
    print(e)

In [ ]:
try:
    logger = Logger(tags_to_set={"y_inner_mtype": ["pd.DataFrame", "pd_multiindex_hier"]})
    logger.fit(y)
except ValueError as e:
    print(e)

#### Input hierárquico

Agora veremos como o modelo se comporta com dados hierárquicos. Note que, nos casos onde o modelo não suporta dados hierárquicos, o sktime tentará convertê-los para um mtype suportado.

In [ ]:
from sktime.utils._testing.hierarchical import _make_hierarchical

y = _make_hierarchical((1,2), max_timepoints=4, min_timepoints=2)
y

In [ ]:
logger = Logger(tags_to_set={"y_inner_mtype" : ["pd.Series"] })
logger.fit(y)

In [ ]:
logger = Logger(tags_to_set={"y_inner_mtype" : ["np.ndarray"] })
logger.fit(y)

In [ ]:
logger = Logger(tags_to_set={"y_inner_mtype" : ["pd.DataFrame"] })
logger.fit(y)

In [ ]:
try:
    logger = Logger(tags_to_set={"y_inner_mtype": ["pd_multiindex_hier"]})
    logger.fit(y)
except ValueError as e:
    print(e)

## Criando um modelo naive

Agora, vamos implementar um modelo simples de previsão, o `CustomNaiveForecaster`, que prevê o valor médio dos últimos n pontos da série temporal.

É um exemplo simples, mas que ilustra bem como criar um forecaster customizado com sktime.

In [ ]:
from tsbook.datasets.retail import SyntheticRetail

dataset = SyntheticRetail("panel")
y_train, y_test  = dataset.load("y_train", "y_test")
y_train

In [ ]:
from sktime.utils.plotting import plot_series

plot_series(
    y_train.loc[0],
    y_train.loc[24],
    labels=[
        "SKU 0",
        "SKU 24",
    ],
)

Abaixo, implementamos o `CustomNaiveForecaster` seguindo as regras do sktime (clique para expandir). Em seguida, explicamos passo a passo.

In [ ]:
# | code-fold: true
from sktime.forecasting.base import BaseForecaster
import pandas as pd


class CustomNaiveForecaster(BaseForecaster):
    """
    A simple naive forecaster

    Parameters
    ----------
    n : int
        Number of past values to use.
    """

    _tags = {
        "requires-fh-in-fit": False,
        "y_inner_mtype": [
            "pd.Series",
        ],
    }

    # Add hyperparameters in init!
    def __init__(self, n=1):
        # 1. Set hyper-parameters
        self.n = n

        # 2. Initialize parent class
        super().__init__()

        # 3. Check hyper-parameters
        assert self.n > 0, "n must be greater than 0"

    def _fit(self, y, X, fh):
        """
        Fit necessary parameters.
        """

        self.value_ = y.iloc[-self.n :].mean()
        return self

    def _predict(self, fh, X):
        """
        Use forecasting horizon and optionally X to predict y
        """

        # During fit, BaseForecaster sets
        # self.cutoff to the latest cutoff time point
        index = fh.to_absolute_index(self.cutoff)
        y_pred = pd.Series(
            index=index,
            data=[self.value_ for _ in range(len(index))],
        )
        y_pred.name = self._y.name

        return y_pred

    # Veremos mais tarde como usar esse método
    @classmethod
    def get_test_params(cls, parameter_set="default"):
        return [
            {"n": 1},
            {"n": 2},
        ]

### Definindo o método `__init__`

O método `__init__` possui 3 etapas:

1. A definição dos hiperparametros e seus atributos com mesmo nome.
2. A chamada do `super().__init__()` para inicializar a classe pai.
3. A validação dos hiperparâmetros.
   


```python
# Add hyperparameters in init!
def __init__(self, n=1):
    # 1. Set hyper-parameters
    self.n = n

    # 2. Initialize parent class
    super().__init__()

    # 3. Check hyper-parameters
    assert self.n > 0, "n must be greater than 0"
```

No caso de algum preprocessamento dos hiperparâmetros no `__init__`, devemos guardar em uma variável com nome diferente do hiperparâmetro. Por exemplo, se tivéssemos interesse em ter um atributo `n` diferente do passado no `__init__`, poderíamos fazer:

```
self._n = n + 1
```

O `self.n` funciona como uma digital do modelo, e deve ser exatamente o que foi passado no `__init__`.

### Definindo o método `_fit`

No método `_fit`, devemos implementar a lógica de ajuste do modelo. No nosso caso, calculamos a média dos últimos `n` valores e armazenamos em `self.value_`.

O `_` após o nome do atributo indica que é um atributo aprendido durante o ajuste, e será retornado quando chamarmos `get_fitted_params()`.

Note que podemos supor que `y` é do tipo definido na tag `y_inner_mtype`, ou seja, uma `pd.Series`.

### Definindo o método `_predict`

No método `_predict`, implementamos a lógica de previsão. Usamos o horizonte de previsão `fh` para determinar os índices futuros e retornamos uma série com o valor previsto para cada ponto no horizonte.

O `fh` é um objeto do tipo `ForecastingHorizon`, que possui o método `to_absolute_index(cutoff)` para converter o horizonte relativo em índices absolutos, considerando o último ponto conhecido (`self.cutoff`).

Retornamos um `pd.Series` com os índices e os valores previstos.

### Usando o `CustomNaiveForecaster`

Agora, já podemos usar o nosso modelo customizado para fazer previsões.

In [ ]:
custom_naive_model = CustomNaiveForecaster()
custom_naive_model.fit(y_train)

Como passamos um dado hierárquico, o sktime converteu automaticamente para `pd.Series`, que é o mtype suportado pelo nosso modelo. Os modelos internos, para cada série, ficam disponíveis em `forecasters_`.

In [ ]:
custom_naive_model.forecasters_

In [ ]:
y_pred = custom_naive_model.predict(fh=y_test.index.get_level_values(-1).unique())

fig, _ = plot_series(
    y_train.loc[0],
    y_pred.loc[0],
    labels=[
        "SKU 0",
        "Previsão SKU 0",
    ],
)
fig.show()

## Testes unitários

O sktime também fornece uma funcionalidade que traz testes unitários prontos para validar se o modelo customizado está funcionando corretamente.

Ele usa os hiperparâmetros retornados pelo método `get_test_params` para criar instâncias do modelo e executar uma série de testes.

In [ ]:
from sktime.utils.estimator_checks import check_estimator


check_estimator(CustomNaiveForecaster, tests_to_exclude=["test_doctest_examples"])